In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import joblib
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split, KFold
import sys
import os
from os import listdir
from os.path import isfile, join
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from tabulate import tabulate
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from deslib.dcs import OLA
# from deslib.dcs import KNORA_U, KNORA_E
from deslib.des import  KNOP, METADES
from deslib.static import SingleBest, StackedClassifier
from deslib.static import StaticSelection
from imblearn.metrics import geometric_mean_score

In [ ]:
FEATURES_SET = {
    "feature": 1,
    "permission": 2,
    "activity": 3,
    "service_receiver": 3,
    "provider": 3,
    "service": 3,
    "intent": 4,
    "api_call": 5,
    "real_permission": 6,
    "call": 7,
    "url": 8
}


def count_feature_set(lines):
    """
    Count how many features belong to a specific set
    :param lines: features in the text file
    :return:
    """
    features_map = {x: 0 for x in range(1, 9)}
    for l in lines:
        if l != "\n":
            set = l.split("::")[0]
            features_map[FEATURES_SET[set]] += 1
    features = []
    for i in range(1, 9):
        features.append(features_map[i])
    return features


In [ ]:
def read(LOAD_DATA=False):
    if LOAD_DATA:
        print("Previous data not loaded. Attempt to read data ...")
        mypath = r"C:\Users\josev\Desktop\ProjetoPessoalNN\GeorgeDarminton\Drebin\MetaData\feature_vectors\feature_vectors"
        onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

        print("Reading csv file for ground truth ...")
        ground_truth = np.loadtxt(r"C:\Users\josev\Desktop\ProjetoPessoalNN\GeorgeDarminton\Drebin\MetaData\sha256_family.csv", delimiter=",", skiprows=1, dtype=str)
        # print ground_truth.shape
        # families = np.unique(ground_truth[:, 1])
        # print families
        # print len(families)

        print("Reading positive and negative texts ...")
        pos = []
        neg = []
        for virus in tqdm(onlyfiles):
            if virus in ground_truth[:, 0]:
                pos.append(virus)
            else:
                #if len(neg) < 5560:
                #if len(neg) < 22240:
                    neg.append(virus)

        print("Extracting features ...")
        x = []
        y = []
        for text_file in tqdm(pos):
            sys.stdin = open("%s/%s" % (mypath, text_file))
            features = sys.stdin.readlines()
            sample = count_feature_set(features)
            x.append(sample)
            y.append(1)

        for text_file in tqdm(neg):
            sys.stdin = open("%s/%s" % (mypath, text_file))
            features = sys.stdin.readlines()
            sample = count_feature_set(features)
            x.append(sample)
            y.append(0)

        print("Data is read successfully:")
        x = np.array(x)
        y = np.array(y)
        print(x.shape, y.shape)

        print("Saving data under data_numpy directory ...")
        np.save(r"C:\Users\josev\Desktop\ProjetoPessoalNN\GeorgeDarminton/x_all.npy", x)
        np.save(r"C:\Users\josev\Desktop\ProjetoPessoalNN\GeorgeDarminton/y_all.npy", y)

        return x, y
    else:
        print("Loading previous data ...")
        x_ = np.load(r"C:\Users\josev\Desktop\ProjetoPessoalNN\GeorgeDarminton/x_all.npy")
        y_ = np.load(r"C:\Users\josev\Desktop\ProjetoPessoalNN\GeorgeDarminton/y_all.npy")
        print(x_.shape, y_.shape)
        # print x == x_, y == y_
        return x_, y_


def map_family_to_category(families):
    out = {}
    count = 1
    for family in families:
        out[family] = count
        count += 1
    return out


if __name__ == "__main__":
    #x, y = read(LOAD_DATA=True)
    x, y = read()

In [ ]:
x_all, y_all = read(LOAD_DATA=False)
x_train, x_test, y_train, y_test = train_test_split(x_all, y_all, test_size=0.2, stratify=y_all, random_state=42)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

In [ ]:
import os
import numpy as np
import pandas as pd

def load_predictions(base_path):
    all_predictions = []
    
    # Percorre todas as pastas dentro do diretório base
    for model_name in os.listdir(base_path):
        model_path = os.path.join(base_path, model_name)
        
        if os.path.isdir(model_path):
            # Para cada modelo (30 execuções)
            model_predictions = []
            for i in range(1, 31):
                file_path = os.path.join(model_path, f'model_{i}.csv')
                
                if os.path.isfile(file_path):
                    # Carrega as previsões do modelo
                    predictions = pd.read_csv(file_path).values.flatten()
                    model_predictions.append(predictions)
            
            # Adiciona as previsões de todas as execuções desse modelo
            all_predictions.append(np.array(model_predictions).mean(axis=0))

    return np.array(all_predictions)

def calculate_instance_hardness(all_predictions, true_labels):
    # Verifica se cada previsão está correta
    correctness_matrix = (all_predictions == true_labels).astype(int)
    # Calcula quantas vezes cada instância foi classificada incorretamente
    instance_hardness = 1 - correctness_matrix.mean(axis=0)
    return instance_hardness

# Caminhos para os modelos balanceados e desbalanceados
unbalanced_path = r'E:\DrebinStudy\Unbalanced'
balanced_path = r'E:\DrebinStudy\Balanced'

# Suponha que os rótulos verdadeiros estão em um CSV
true_labels = pd.read_csv(r'E:\DrebinStudy\true_labels.csv').values.flatten()

# Carrega as previsões
unbalanced_predictions = load_predictions(unbalanced_path)
balanced_predictions = load_predictions(balanced_path)

# Calcula Instance Hardness
unbalanced_hardness = calculate_instance_hardness(unbalanced_predictions, true_labels)
balanced_hardness = calculate_instance_hardness(balanced_predictions, true_labels)

# Cria DataFrames para salvar os resultados
df_unbalanced_hardness = pd.DataFrame({
    'Instance': np.arange(len(true_labels)),
    'Instance Hardness': unbalanced_hardness,
    'True Label': true_labels
})

df_balanced_hardness = pd.DataFrame({
    'Instance': np.arange(len(true_labels)),
    'Instance Hardness': balanced_hardness,
    'True Label': true_labels
})

# Salva os resultados em CSV
df_unbalanced_hardness.to_csv(r'E:\DrebinStudy\Unbalanced_Instance_Hardness.csv', index=False)
df_balanced_hardness.to_csv(r'E:\DrebinStudy\Balanced_Instance_Hardness.csv', index=False)

print("Instance Hardness calculada e salva com sucesso!")